In [63]:
import pandas as pd 
df = pd.read_csv("sap_storing.csv") 

__1.1 Target variabele bepalen__

De target variabele waarnaar we willen gaan voorspellen wordt de eindtijd van de storing. 

__1.2 Inhoud en betekenis target variabele__


We willen voor elk ongeval voorspellen hoe lang de storing gaat duren, hiervoor willen we de eindtijd van de storing voorspellen. Om te kijken naar hoe snel eerdere problemen opgelost zijn hebben we een nieuwe variabele nodig: namelijk de tijd tussen het tijdstip van de melding van een ongeval en het tijdstip waarop het de storing verholpen is. 

In [69]:
df[['stm_sap_storeindtijd', 'stm_sap_meld_ddt']]

,stm_sap_storeindtijd,stm_sap_meld_ddt
0,09:00:00,02/01/2006 09:00:00
1,13:26:00,02/01/2006 12:35:00
2,17:20:00,02/01/2006 16:40:00
3,22:36:00,02/01/2006 22:30:00
4,00:00:00,02/01/2006 11:23:00
...,...,...
74,09:30:00,11/05/2013 08:06:00
75,10:44:00,11/05/2013 09:21:00
76,10:44:00,11/05/2013 09:21:00
77,15:15:00,20/08/2016 14:15:17


__1.3 Target variabele geschikt maken voor model__

In [73]:
type(df.stm_sap_storeinddatum)

pandas.core.series.Series

In [74]:
type(df.stm_sap_meld_ddt)

pandas.core.series.Series

????

In [75]:
dates_list = [datetime.strptime(date, '"%d-%m-%y %H:%M:%S"').date() for date in data.stm_sap_meld_ddt]

ValueError: time data '02/01/2006 09:00:00' does not match format '"%d-%m-%y %H:%M:%S"'

In [59]:
FMT = '%H:%M:%S'
from datetime import datetime
df['diff'] = datetime.strptime(df['stm_sap_meld_ddt'], FMT) - datetime.strptime(df['stm_sap_storeindtijd'], FMT).df['fh']

TypeError: strptime() argument 1 must be str, not Series